# <span style="color:orange">31/05 - Pair Regresión 6- Alicia y Lidia</span>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve,roc_auc_score, classification_report
from sklearn.metrics import accuracy_score 
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean, std

plt.rcParams["figure.facecolor"] = "w"

In [2]:
import warnings
warnings.filterwarnings('ignore')

**Elegimos los datos del mejor modelo obtenido anteriormente.**

In [3]:
df = pd.read_csv('anime_codi_estand.csv', index_col=0)
df

,title,episodes,members,popularity,ranked,score,score_class
0,4175,25.0,2389,141,25.0,8.82,1
1,11065,22.0,2389,28,24.0,8.83,1
2,7324,13.0,2389,98,23.0,8.83,1
3,3331,2.0,2389,4,1.0,6.41,1
4,6414,1.0,2389,502,22.0,8.83,1
...,...,...,...,...,...,...,...
19306,3232,13.0,2389,843,1070.0,7.73,1
19307,3363,2.0,2389,1292,1071.0,7.73,1
19308,3486,19.0,2389,1314,1073.0,7.73,1
19309,5264,1.0,2389,1469,1077.0,7.73,1


**Recordamos que en 'score_class' el 0 es 'puntuación_baja' y el 1 'puntuación_alta'**

### 1. Aplica algún método de validación cruzada obteniendo las metricas de los diferentes ajustes realizados, así como la media de todos los ajustes.


**Decidimos usar el método de k-pliegues(K fold cross validation) que es el método más utilizado y que arroja mejores resultados. Para cada pliegue  se realiza el ajuste del modelo y se testea el modelo para cada conjunto de test de cada uno de ellos.**

In [4]:
x = df.drop(['score_class'], axis = 1)
y = df['score_class']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(12700, 6) (12700,)
(3175, 6) (3175,)


In [5]:
cv = KFold(n_splits=10, random_state=42, shuffle=True) #Hemos decidido hacer 10 pliegues.
# create model
model = LogisticRegression(n_jobs=-1, max_iter = 1000)
# evaluate model
scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

print('Los valores de la exatitud han sido :',scores)
print('La exactitud media de la validación cruzada: %.3f (%.3f)' % (mean(scores), std(scores)))

Los valores de la exatitud han sido : [0.9023622  0.89685039 0.90787402 0.90866142 0.89133858 0.9015748
 0.91811024 0.8992126  0.90708661 0.90629921]
La exactitud media de la validación cruzada: 0.904 (0.007)


**Hemos obtenido los valores exactos para cada pliegue y la media del total de ellos que vemos que es bastante buena porque es cercano a 1. También al ver la desviación estandar que es tan pequeñita (0,007) podemos concluir que es un buen modelo**

### 2. Utiliza el gridsearch definiendo los diferentes hiperparámetros y configuraciones de los mismos.

In [10]:
#Podríamos ir probando por cada hiperparámetro pero es más sencillo utilizar este método y que calcule el mejor.
grid_values = {'penalty': ['l1','l2'], 'C': [0.001,0.01,0.1,1,10,100,1000], 'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'] }


lr = LogisticRegression()
lr_gridsearchcv = GridSearchCV(estimator = lr, param_grid = grid_values, cv = 5, verbose=1, n_jobs = -1)

lr_gridsearchcv.fit(x_train, y_train)

# sacamos el mejor modelo de todos los fiteados
print(f'Los mejores parámetros para nuestro modelo son: {lr_gridsearchcv.best_params_}')


Fitting 5 folds for each of 70 candidates, totalling 350 fits


/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selectio

Los mejores parámetros para nuestro modelo son: {'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}


**Ahora conocemos los mejores parámetros para nuestra regresión loigistica que son:**
- *penalty* usaremos l1
- *solver* el mejor es liblinear
- *C* escogemos 1000

### 3. Una vez se tengan los mejores hiperparámetros, implementa la regresión logísitca con estos hiperparámetros y obtenen las métricas relevantes.

**Esperamos tener mejor regresión logistica**🤞🏻

In [7]:
log_reg = LogisticRegression(n_jobs=-1, max_iter = 1000, C =1000, penalty= 'l1', solver ='liblinear')
# ajustamos el modelo
log_reg.fit(x_train,y_train)
# obtenemos las predicciones para el conjunto de entrenamiento
y_pred_train = log_reg.predict(x_train)
# obtenemos las predicciones para el conjunto de test
y_pred_test = log_reg.predict(x_test)


train_score_lr = accuracy_score(y_train, y_pred_train)
print('Para el conjunto train:---------------')
print('El valor de accuracy  es ',train_score_lr)

print('Para el conjunto test:---------------')
test_score_lr = accuracy_score(y_test, y_pred_test)
print('El valor de accuracy  es ',test_score_lr)

Para el conjunto train:---------------
El valor de accuracy  es  0.9966929133858268
Para el conjunto test:---------------
El valor de accuracy  es  0.9965354330708661


**Al incluir los mejores hiperparámetros obtenidos anteriormente, vemos que aunque nuestro modelo ya era bastante ajustado a una regresión logística porque era 0.904, hemos podido mejorarlo a 0,997, es decir, casi perfecto porque es cercano a 1**

**Parece que elegimos un buen datasets** 😜👏🏻